In [110]:
import numpy as np
import pandas as pd

In [76]:
def update_df_from_edit(df: pd.DataFrame, edits: pd.DataFrame) -> pd.DataFrame:
    out = df.copy().reset_index(drop=True)
    for rid, col, val in edits[["row_id","column","new_value"]].itertuples(index=False):
        out.loc[int(rid), col] = val
    return out

In [142]:
df = pd.read_parquet("w_incl_25_06_dedup_v4_corrected_07_3.parquet.gzip")

In [143]:
edits = pd.read_csv("edits/edits_06_25_08_1.csv")

In [144]:
if not pd.api.types.is_datetime64_any_dtype(edits["timestamp_utc"]):
    edits["timestamp_utc"] = pd.to_datetime(edits["timestamp_utc"], utc=True)

idx = edits.groupby(["row_id", "pole_id", "column"])["timestamp_utc"].idxmax()
edits_latest = (edits.loc[idx]
                   .sort_values(["row_id", "pole_id", "column"])
                   .reset_index(drop=True))

In [145]:
updated_df = update_df_from_edit(df, edits_latest)

In [146]:
mask = updated_df["misplacement"].isna() & updated_df["rail_incl_corrected"].notna() & updated_df["rail_incl_corrected"].notna() & updated_df.pole_id.isin(edits.pole_id.to_list())
updated_df.loc[mask, "misplacement"] = (
    updated_df["shoulder_height"] * np.tan(np.deg2rad(updated_df["rail_incl_corrected"]))
)

In [147]:
updated_df.to_parquet("w_incl_25_06_dedup_v4_corrected_08_1.parquet.gzip")